In [4]:
import sqlite3
import numpy as np
import json
import pandas as pd
import mapclassify
import csv
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'firefox'

states = []
statesCode = {}
statesCovidData = {}
statesPopulation = {}

with open("stateData.json", "r") as read_file:
    print("Started converting JSON string document to Python dictionary")
    indiaStateData = json.load(read_file)
    states = list(indiaStateData.keys())
    for state in states:
        statesCode[state] = indiaStateData[state]["statecode"]

with open("indiaCovidData.json", "r") as covid_file:
    print("Started converting JSON string document to Python dictionary")
    indiaCovidData = json.load(covid_file)
    for key,value in statesCode.items():
        try:
            #print(key, value, indiaCovidData[value]["total"])
            statesCovidData[key] = indiaCovidData[value]["total"]
        except KeyError: 
            continue
    #print(statesCovidData)

tsv_file = open("india.state-population.tsv")
read_tsv = csv.reader(tsv_file, delimiter="\t")

for row in read_tsv:
    statesPopulation[row[0]] = row[1]
#print(statesPopulation)

state_id_map = {}
india_states = json.load(open("india-all-states.geo.json"))
#print(india_states['features'][1])

for feature in india_states['features']:
    feature['id'] = feature['properties']['ID_1']
    state_id_map[feature['properties']['NAME_1']] = feature['id']
print(state_id_map)

tsv_file.close()
    
#opens the database connection
def openDatabase():
    conn = sqlite3.connect('indiaCovidData.db')
    return conn

#deletes the table
def dropTable():
    conn = openDatabase()
    conn.execute('''DROP TABLE indiaStateCovidData;''')

#creates the table
def createTable() :
    conn = openDatabase()
    conn.execute('''CREATE TABLE indiaStateCovidData  (ID INT PRIMARY KEY,
                                                       NAME TEXT NOT NULL,
                                                       POPULATION INT, 
                                                       NAMECODE TEXT,
                                                       CONFIRMED INT,
                                                       DECEASED INT,
                                                       RECOVERED INT,
                                                       TESTED INT);''')
    conn.commit()
    print ("Table created successfully");
    conn.close()

#initially populates the table with contactlist present in the file
def fillTable() :
    conn = openDatabase()
    print("Filling Table with initial values")
    states.sort()
    i = 0
    for state in states:
        try:
            if(state == "Odisha"):
                stateIdMap = state_id_map["Orissa"]
            else:
                stateIdMap = state_id_map[state]
            params = (stateIdMap, state, statesPopulation[state], statesCode[state], statesCovidData[state]["confirmed"],
                 statesCovidData[state]["deceased"], statesCovidData[state]["recovered"],
                 statesCovidData[state]["tested"])
            conn.execute("INSERT INTO indiaStateCovidData (ID, NAME, POPULATION, NAMECODE, CONFIRMED, DECEASED, RECOVERED, TESTED) VALUES (?, ?, ?, ?, ?, ?, ?, ?)", params);
        except KeyError:
            continue
    conn.commit()
    conn.close()

def getDataFrame() :
    conn = openDatabase()
    print("Creating dataframe")
    cursor = conn.execute("SELECT * FROM indiaStateCovidData")
    state_id = []
    name = []
    population = []
    namecode = []
    confirmed = []
    deceased = []
    recovered = []
    tested = []
    for row in cursor:
        if(row[1] == "Oda"):
            print(row)
        state_id.append(row[0])
        name.append(row[1])
        population.append(row[2])
        namecode.append(row[3])
        confirmed.append(row[4])
        deceased.append(row[5])
        recovered.append(row[6])
        tested.append(row[7])
    data = {"id": state_id, "Name": name, "Population": population, "Namecode": namecode, "Confirmed": confirmed,
         "Deceased": deceased, "Recovered": recovered, "Tested":tested}
    conn.close()
    return data

dropTable()
createTable()
fillTable()
df = pd.DataFrame(getDataFrame())
df['ConfirmedScale'] = np.log10(df['Confirmed'])
print(df)

fig = px.choropleth(df, 
                    locations='id', 
                    geojson=india_states, 
                    scope='asia',
                    color="ConfirmedScale",
                    hover_name='Name',
                    hover_data=['Confirmed'])
fig.update_geos(fitbounds="locations", visible=False)
fig.show()

    
    




Started converting JSON string document to Python dictionary
Started converting JSON string document to Python dictionary
{'Andaman and Nicobar': 1, 'Telangana': 36, 'Andhra Pradesh': 2, 'Arunachal Pradesh': 3, 'Assam': 4, 'Bihar': 5, 'Chandigarh': 6, 'Chhattisgarh': 7, 'Dadra and Nagar Haveli': 8, 'Daman and Diu': 9, 'Delhi': 10, 'Goa': 11, 'Gujarat': 12, 'Haryana': 13, 'Himachal Pradesh': 14, 'Jammu and Kashmir': 15, 'Jharkhand': 16, 'Karnataka': 17, 'Kerala': 18, 'Lakshadweep': 19, 'Madhya Pradesh': 20, 'Maharashtra': 21, 'Manipur': 22, 'Meghalaya': 23, 'Mizoram': 24, 'Nagaland': 25, 'Orissa': 26, 'Puducherry': 27, 'Punjab': 28, 'Rajasthan': 29, 'Sikkim': 30, 'Tamil Nadu': 31, 'Tripura': 32, 'Uttar Pradesh': 33, 'Uttaranchal': 34, 'West Bengal': 35}
Table created successfully
Filling Table with initial values
Creating dataframe
    id               Name   Population Namecode  Confirmed  Deceased  \
0    2     Andhra Pradesh   49,577,103       AP     120390      1213   
1    3  Aruna